In [5]:
from random import randrange

p = 2
m = 2
pm = p^m

n = 425 #70 #70 #530 #425 #70 # 30
d = 85 #9 #9 #185 #148 #9
t = 42 #floor((d-1)/2) #2

def GB_k(n,d):
    temp = 0
    for j in range(0,d):
        temp = temp + binomial(2*n,j)
    return (4^n)/(3*(temp-1)+1)/1.0

k =  229 #552 #22 #13 #179 #145 # 22 #13
k1 = 33 #7 #7# 11 #7
k2 = 2*(k-k1)

print("Parameters (n, k, k1, k2, d, t) =", (n,k,k1,k2,d,t))

R = IntegerModRing(4)
R2 = IntegerModRing(2)

A12 = matrix(R,k1,k2)
for i in range(0,k1):
    for j in range(0,k2):
        A12[i,j] = R.random_element()

A13 = matrix(R,k1,n-k1-k2)
for i in range(0,k1):
    for j in range(0,n-k1-k2):
        A13[i,j] = R.random_element()

G_k1 = ((matrix.identity(R,k1)).augment(A12)).augment(A13)

A23 = matrix(R,k2,n-k1-k2)
for i in range(0,k2):
    for j in range(0,n-k1-k2):
        A23[i,j] = 2*randrange(0,2)

G_k2 = (matrix(R,k2,k1).augment(2*matrix.identity(R,k2))).augment(A23)

Gpub = G_k1.stack(G_k2)


def RandList(n):
    count = 0
    randlist = []
    while count < n:
        temp = randrange(0,n)+1
        countrepeat = 0
        for i in range(0,count):
            if temp == randlist[i]:
                countrepeat = 1
        if countrepeat == 0:
            randlist = randlist + [temp]
            count = count +1
    return randlist

def Encode(n,k1,k2,Gpub):
    mu = matrix(R,1,k1+k2)
    for j in range(0,k1):
        mu[0,j] = R.random_element()
    for j in range(k1,k1+k2-1):
        mu[0,j] = R(randrange(0,2))
    return mu,mu*Gpub

def LeeWt(v):
    Lw = ZZ(0)
    for j in range(0,v.ncols()):
        Lw = Lw + ZZ(min(v[0,j],pm-v[0,j]))
    return Lw

def RandLeeError(n,t):
    e = matrix(R,1,n)
    count = 0
    while count < n:
        if LeeWt(e) < t:
            e[0,count] = R.random_element()
        elif LeeWt(e) == t:
            e[0,count] = 0
        else:
            count = count - 1
            e[0,count] = R.random_element()
        count = count + 1
    sigma = Permutation(RandList(n))
    P = sigma.to_matrix();
    return e*P

def Mod_p(M,p,k1):
    IRp = IntegerModRing(p)
    NewM = Matrix(IRp,k1,M.ncols())
    for i in range(0,k1):
        for j in range(0,M.ncols()):
            NewM[i,j] =IRp(M[i,j])
    return NewM

Gpub1k1 = Mod_p(Gpub,p,k1)
M = matrix(GF(2), Gpub1k1)
C = codes.LinearCode(M)

#print(C, "has minimum distance = ", C.minimum_distance() )

Parameters (n, k, k1, k2, d, t) = (425, 229, 33, 392, 85, 42)


In [6]:
earlytime = cputime(subprocesses=True)
totaltime = earlytime
iteration = 2 #10
for ite in range(iteration):
    mu,y = Encode(n,k1,k2,Gpub)
    e = RandLeeError(n,t)
    c = y + e

    #print("y = ", y)
    #print("e = ", e)
    #print("c = ", c)

    cprime = Mod_p(c,p,1)
    eprime = Mod_p(e,p,1)
    yprime = Mod_p(y,p,1)

    print("yprime = ", yprime)
    print("eprime = ", eprime)
    #print("cprime = ", cprime)

    bt=cputime(subprocesses=True)    

    print("=========================Step 1==========================")

    c_vec = []
    for j in range(0,n):
      c_vec = c_vec + [cprime[0,j]]

    from sage.coding.information_set_decoder import LeeBrickellISDAlgorithm #InformationSetAlgorithm #
    A = LeeBrickellISDAlgorithm(C, (0,t)) #codes.decoders.LinearCodeNearestNeighborDecoder(C)

    r = vector(GF(2), c_vec)
    
    print("r =", r)
    
    y_dec = A.decode(r)

    et=cputime(subprocesses=True)-bt
    totaltime = totaltime + et
    #print("CPU time =", et)
    
    #print("decoded codeword is ", y_dec)
    
    e_dec = y_dec - cprime[0]
    #print("encrypted error is ", e_dec)
    print("current iteration:", ite)
    print(y_dec == yprime[0])
    print(e_dec == eprime[0])

    print("=========================================================")
    
print("average time taken = ", RR(totaltime-earlytime)/RR(iteration))

yprime =  [0 1 1 1 1 0 1 1 0 0 1 0 1 0 0 1 1 1 0 1 1 0 1 0 1 0 0 1 1 0 1 1 1 0 0 1 1 1 1 1 0 1 0 0 1 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 1 1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 1 0 0 1 1 0 0 1 1 0 0 0 1 0 0 1 1 1 0 1 0 0 0 1 0 1 0 1 0 1 0 0 0 1 0 1 0 0 1 1 0 0 1 1 1 0 1 0 0 0 1 1 1 1 1 1 1 0 1 1 0 1 0 0 1 0 0 0 0 1 0 0 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 0 1 1 0 1 1 1 1 1 0 1 0 0 1 1 0 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 1 0 1 0 1 1 1 1 1 1 1 0 0 0 1 0 1 0 1 1 1 1 0 0 0 1 1 0 1 1 1 0 0 1 1 0 0 0 1 1 0 0 0 1 1 0 1 1 0 1 1 1 1 0 0 0 1 0 0 1 1 0 1 0 1 0 0 0 1 1 0 1 1 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 1 0 1 1 1 1 0 0 0 1 1 1 1 1 0 0 1 1 1 0 1 0 0 1 0 0 1 1 1 0 0 0 1 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 1 1 0 0 1 1 0 0 0 0 1 0 1 1 1 0 1 1 0 0 1 1 0 0 0 1 0 1 0 1 1 1 1 1 0 0 0 0 0 1 1 1 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 1 1]
eprime =  [0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1

In [7]:
print(n,k,k1,k2)

print("=========================Step 2==========================")

x_p = matrix(R,1,k1)
for j in range(0,k1):
  x_p[0,j] = R(yprime[0,j])

eprime_p = matrix(R,1,n)
for j in range(0,n):
  eprime_p[0,j] = R(eprime[0,j])

cpprime = c - x_p*G_k1 - eprime_p

Gpubprimek1 = matrix(GF(2),Gpub1k1)
Gpubprimek2 = matrix(GF(2),k2,n)
for i in range(0,k2):
    for j in range(0,n):
        Gpubprimek2[i,j] = GF(2)(ZZ(G_k2[i,j])/ZZ(2))

Gpubprime = Gpubprimek1.stack(Gpubprimek2)
Mprime = matrix(GF(2), Gpubprime)
Cprime = codes.LinearCode(Mprime)



c_hat = matrix(GF(2),1,n)
for j in range(0,n):
    c_hat[0,j] = GF(2)(ZZ(cpprime[0,j])/ZZ(2))


c_hat_vec = []
for j in range(0,n):
  c_hat_vec = c_hat_vec + [c_hat[0,j]]

print(c_hat)

bt=cputime(subprocesses=True)

from sage.coding.information_set_decoder import LeeBrickellISDAlgorithm
Aprime = LeeBrickellISDAlgorithm(Cprime, (0,t))

rprime = vector(GF(2), c_hat_vec)

y_hat_dec = Aprime.decode(rprime)

et=cputime(subprocesses=True)-bt

print("time used =", et)

print("decoded codeword is ")
print(y_hat_dec)

e_hat_dec = y_hat_dec - c_hat[0]
print("encrypted error is ")
print(e_hat_dec)

b2t=cputime(subprocesses=True)

e_comp = matrix(R,1,n)
for j in range(0,n):
    e_comp[0,j] = R(ZZ(e_dec[j]) + ZZ(2)*ZZ(e_hat_dec[j]) )

e2t=cputime(subprocesses=True)-b2t

print("time used =", e2t)
    
print(e)
print(e_comp)

print(e_comp == e)

codeword = c - e_comp

print("codeword is equal to y?")
print(codeword == y)


425 229 33 392
=========================Step 2==========================
[0 1 1 0 0 1 1 0 0 1 0 1 1 1 1 0 0 1 1 0 0 1 1 0 1 0 0 0 0 1 0 0 0 1 1 0 1 1 0 1 0 0 0 0 0 0 1 1 0 1 1 1 1 0 1 0 0 0 0 1 1 0 0 0 1 0 1 1 1 0 0 0 1 0 1 0 1 1 1 0 0 1 1 0 1 1 0 0 0 0 0 0 0 1 1 0 1 1 1 0 0 0 1 1 0 1 0 0 1 0 0 0 1 0 1 0 1 1 0 1 1 1 1 0 0 0 1 0 1 1 0 1 1 0 0 1 0 0 0 1 1 1 1 1 1 0 1 0 1 0 0 0 1 1 1 0 1 1 0 0 0 1 1 0 0 0 0 1 1 0 0 1 1 0 1 0 0 1 1 0 0 1 1 1 0 1 1 1 1 1 0 1 0 0 1 1 1 0 1 0 0 1 1 0 0 1 0 1 0 1 1 1 0 0 1 1 0 1 0 0 0 0 1 0 1 1 1 1 1 0 0 0 1 1 0 1 1 1 0 1 0 0 0 1 0 1 0 1 1 1 1 0 0 0 0 0 0 1 0 1 1 1 1 0 0 0 1 1 0 0 0 1 0 0 0 0 1 0 1 0 1 0 0 1 1 0 0 0 1 0 0 1 0 0 1 1 1 1 1 1 0 1 1 0 1 0 1 0 0 1 0 1 0 1 1 1 0 1 1 0 1 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 0 0 1 0 0 1 1 1 1 1 0 1 1 0 0 0 0 0 1 1 1 0 0 0 1 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 1 0 0 0 0 1 1 1 0 0 1 0 1 1 0 0 1]


ZeroDivisionError: float division by zero

In [ ]:
b3t=cputime(subprocesses=True)  
print(Cprime, "has minimum distance = ", Cprime.minimum_distance())
e3t=cputime(subprocesses=True)-b3t

print("time used =", e3t)